In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/persona-chat/spc_dataset/

Mounted at /content/drive
/content/drive/MyDrive/persona-chat


In [4]:
# split
import json
from sklearn.model_selection import train_test_split

with open('spc_dataset.json', 'r') as file:
    data = json.load(file)

train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

with open('train_spc_dataset.json', 'w') as file:
    json.dump(train_data, file)
with open('val_spc_dataset.json', 'w') as file:
    json.dump(val_data, file)

In [ ]:
from transformers import GPTNeoXTokenizerFast

tokenizer = GPTNeoXTokenizerFast.from_pretrained('EleutherAI/gpt-neox-20b')

def load_data(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

def tokenize_spc(data, tokenizer):
    tokenized_data = []
    for item in data:
        history = "<History>"
        user1_persona = f"<Persona 1> {item['user1_persona']}"
        user2_persona = f"<Persona 2> {item['user2_persona']}"
        for convo in item['conversations']:
            user_text = f"<User> {convo['user']}"
            bot_text = f"<Bot> {convo['bot']}"
            conversation = f"{user1_persona} {user2_persona} {history} {user_text} {bot_text}"
            conversation_sequence = tokenizer.encode(conversation, truncation=True, max_length=2048)
            tokenized_data.append(conversation_sequence)
            history += f" {convo['user']} {convo['bot']} "
    return tokenized_data

data1 = load_data('spc_dataset.json')
tokenized_data1 = tokenize_spc(data1, tokenizer)

for encoded_input in tokenized_data1[:5]:
    print(encoded_input)
    decoded_text1 = tokenizer.decode(encoded_input)
    print(decoded_text1)